# UCF101 Handler - For the Paper

## Source:
https://www.crcv.ucf.edu/data/UCF101.php<br>

## RGB Images:
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_jpegs_256.zip.001<br>
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_jpegs_256.zip.002<br>
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_jpegs_256.zip.003<br>

## Optical Flow:
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_tvl1_flow.zip.001<br>
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_tvl1_flow.zip.002<br>
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_tvl1_flow.zip.003<br>

In [ ]:
import numpy as np
import cv2, random, os, math, shutil, time, warnings, glob, \
  threading, pickle
import pandas as pd
from zipfile import ZipFile
from scipy import ndimage
from multiprocessing import Pool
from multiprocessing import Process

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, \
  EarlyStopping, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.layers import Input, Average, LSTM, \
  Dense, Dropout, Activation, Flatten, SpatialDropout2D, Reshape, \
  GlobalAveragePooling2D, Conv2D, TimeDistributed, \
  MaxPooling2D, BatchNormalization, GlobalAveragePooling1D
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Convolution2D, MaxPooling3D, ConvLSTM2D
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
import tensorflow.keras.callbacks
import tensorflow.keras.utils

from sklearn.model_selection import train_test_split

%matplotlib inline
def warn(*args, **kwargs): pass
warnings.warn = warn
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
np.random.seed(1)

In [ ]:
class threadsafe_iterator:
  def __init__(self, iterator):
    self.iterator = iterator
    self.lock = threading.Lock()

  def __iter__(self):
    return self

  def __next__(self):
    with self.lock:
      return next(self.iterator)

def threadsafe_generator(func):
  """Decorator"""
  def gen(*a, **kw):
    return threadsafe_iterator(func(*a, **kw))
  return gen
    
def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

def ComputeTVL1(prev, current, bound=15):
  """Compute the TV-L1 optical flow."""
  TVL1 = cv2.optflow.DualTVL1OpticalFlow_create()
  flow = TVL1.calc(prev, current, None)
  assert flow.dtype == np.float32
  flow = (flow + bound) * (255.0 / (2*bound))
  flow = np.round(flow).astype(int)
  flow[flow >= 255] = 255
  flow[flow <= 0] = 0
  return flow

def AugmentFrame(frame, inputShape, rotate=10, shearRange=5,
    zoom=5, horizontalFlip=False, translationRange=5, 
    changeBrightness=True, cval=255):
  assert frame is not None
  rows, cols, ch = inputShape   
  if (random.random() >= 0.5 and rotate): # Rotate
    angle = random.randint(-rotate, rotate)
    frame = frame.astype(np.uint8)
    frame = ndimage.rotate(frame, angle, cval=255)
  if (random.random() >= 0.5 and translationRange): # Translation
    trX = translationRange*np.random.uniform()-translationRange/2
    trY = translationRange*np.random.uniform()-translationRange/2
    transM = np.float32([[1, 0, trX],[0, 1, trY]])
    frame = cv2.warpAffine(frame, transM, (cols, rows), 
                           borderValue=(cval, cval, cval))
  if (random.random() >= 0.5 and shearRange): # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])
    pt1 = 5+shearRange*np.random.uniform()-shearRange/2
    pt2 = 20+shearRange*np.random.uniform()-shearRange/2
    pts2 = np.float32([[pt1, 5], [pt2, pt1], [5, pt2]])
    shearM = cv2.getAffineTransform(pts1, pts2)
    frame = cv2.warpAffine(frame, shearM, (cols, rows), 
                           borderValue=(cval, cval, cval))
  if (ch == 3 and random.random() >= 0.55 and changeBrightness): # Brightness
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    randomBright = 0.25 + np.random.uniform()
    frame[:,:,2] = frame[:,:,2] * randomBright
    frame = cv2.cvtColor(frame, cv2.COLOR_HSV2RGB)
  if (random.random() >= 0.60 and horizontalFlip): # H-Flip
    frame = cv2.flip(frame, 1)
  if (random.random() >= 0.65 and zoom): # Zoom:
    width, height = inputShape[:2]
    centerX, centerY = int(height/2), int(width/2)
    radiusX, radiusY = int(zoom*height/100), int(zoom*width/100)
    minX, maxX = centerX-radiusX, centerX+radiusX
    minY, maxY = centerY-radiusY, centerY+radiusY
    cropped = frame[minX:maxX, minY:maxY]
    frame = cv2.resize(frame, inputShape[:2],
      interpolation = cv2.INTER_AREA)
  return frame

# UCF101Dataset

In [ ]:
class VideosDataset(tensorflow.keras.utils.Sequence):
  def __init__(self, datasetPath, datasetKeyword, splits=(0.8, 0.1, 0.1),
               batchSize=64, inputShape=(100, 100, 3), isGray=False, 
               storeCSV=True, storageDir="", debug=True, extension="avi", 
               shuffle=True, maxInMemory=5000, segmentation=1, packets=1):
    self.datasetPath = datasetPath         # Videos Path
    self.splits = splits     # Train - Test - Validation
    self.storeCSV = storeCSV # Store CSV files?
    self.datasetKeyword = datasetKeyword
    self.storageDir = storageDir
    self.debug = debug
    self.extension = extension
    self.maxInMemory = maxInMemory
    self.shuffle = shuffle
    self.inputShape = inputShape
    self.batchSize = batchSize
    self.isGray = isGray
    self.segmentation = segmentation
    self.packets = packets
    self.metadata = None
    
    if (self.storeCSV):
      self.metadata = os.path.join(storageDir, 
        self.datasetKeyword + "_All_Metadata.csv")
      self.trainMetadata = os.path.join(storageDir, 
        self.datasetKeyword + "_Train_Metadata.csv")
      self.testMetadata = os.path.join(storageDir, 
        self.datasetKeyword + "_Test_Metadata.csv")
      self.validationMetadata = os.path.join(storageDir, 
        self.datasetKeyword + "_Validation_Metadata.csv")

    self.framesDir = os.path.join(storageDir, 
      self.datasetKeyword + "_Frames")

    self.flowsDir = os.path.join(storageDir, 
      self.datasetKeyword + "_Flows")

    self.trainRecords = []
    self.testRecords = []
    self.validationRecords = []
  
    self.records = []
    self.classes = []
    self.noOfClasses = 0
    
    self.videosInfo = []
    self.classesVideos = {}

  def GetNoOfClasses(self):
    return self.noOfClasses

  def ProcessVideosMetadata(self):
    if(self.debug): print("[STARTED] ProcessVideosMetadata.")
    
    videoNames = glob.glob(self.datasetPath + "/*." + self.extension)
    if(self.debug): print("[INFO] There are %d videos." % len(videoNames))
    
    if (len(videoNames) <= 0): 
      if(self.debug): 
        print("[WARNING] There are no files in this path.")
        print("[COMPLETED] processVideosMetadata.")
      return

    for i, videoPath in enumerate(videoNames):
      videoPath = videoPath.strip()
      extension = videoPath.split(".")[-1]
      if (extension != self.extension): continue
      videoName = os.path.basename(videoPath)
      splitName = videoName.split("_")
      className = splitName[1].strip()
      g = splitName[2].strip()
      c = splitName[3].strip().split(".")[0]
      gc = g + "_" + c

      record = (i + 1, videoName, className, g, c, gc, extension)

      if (className not in self.classes):
        self.classes.append(className)
        self.classesVideos[className] = []
      self.classesVideos[className].append(record)
      self.records.append(record)

    self.classes = sorted(list(set(self.classes)))
    self.noOfClasses = len(self.classes)

    if (self.storeCSV):
      if(self.debug): print("[INFO] There are %d records." % len(self.records))
      df = pd.DataFrame(self.records, 
        columns=["number", "video_name", "class_name",
                "g", "c", "g_c", "extension"])
      df.to_csv(self.metadata, index=False)
    if(self.debug): print("[COMPLETED] ProcessVideosMetadata.")

  def SplitDataset(self):
    if(self.debug): print("[STARTED] SplitDataset.")
    columns = ["number", "video_name", "class_name", "g", "c", "g_c", "extension"]
    df = pd.read_csv(self.metadata)
    self.records = list(df.values)
    self.classes = list(sorted(set(df["class_name"])))
    if(self.debug): print("[INFO] There are", len(self.classes), "classes.")
    for cls in self.classes:
        self.classesVideos[cls] = []
    for record in self.records:
      self.classesVideos[record[2]].append(record)
    
    trainRatio = self.splits[0]
    testRatio = self.splits[1]
    validationRatio = self.splits[2]
    
    for className in self.classes:
      shuffledRecords = self.classesVideos[className][:]
      random.shuffle(shuffledRecords)
      classDict = {}
      for record in shuffledRecords:
        if (record[3] not in classDict.keys()):
            classDict[record[3]] = []
        classDict[record[3]].append(record)
      for k in classDict.keys():
        L = len(classDict[k])
        trainIndex = math.ceil(L * trainRatio)
        testIndex = math.floor(L * testRatio)
        self.trainRecords.extend(classDict[k][:trainIndex])
        self.testRecords.extend(classDict[k][trainIndex:trainIndex+testIndex])
        self.validationRecords.extend(classDict[k][trainIndex+testIndex:])

    if(self.debug): 
      print("[INFO] There are", len(self.trainRecords), "videos for training.")
      print("[INFO] There are", len(self.validationRecords), "videos for validation.")
      print("[INFO] There are", len(self.testRecords), "videos for testing.")

    if (self.storeCSV):
      df = pd.DataFrame(self.trainRecords, columns=columns)
      df.to_csv(self.trainMetadata, index=False)
      df = pd.DataFrame(self.testRecords, columns=columns)
      df.to_csv(self.testMetadata, index=False)
      df = pd.DataFrame(self.validationRecords, columns=columns)
      df.to_csv(self.validationMetadata, index=False)
    if(self.debug): print("[COMPLETED] SplitDataset.")
    
  def ReadVideosMetadata(self):
    if(self.debug): print("[STARTED] ReadVideosMetadata.")
    df = pd.read_csv(self.metadata)
    self.records = list(df.values)
    self.classes = list(sorted(set(df["class_name"])))
    for record in self.records:
      if (record[2] not in self.classesVideos.keys()):
        self.classesVideos[record[2]] = []
      self.classesVideos[record[2]].append(record)
    df = pd.read_csv(self.trainMetadata)
    self.trainRecords = list(df.values)
    df = pd.read_csv(self.testMetadata)
    self.testRecords = list(df.values)
    df = pd.read_csv(self.validationMetadata)
    self.validationRecords = list(df.values)
    self.noOfClasses = len(self.classes)
    if(self.debug): 
      print("[INFO] There are", len(self.trainRecords), "videos for training.")
      print("[INFO] There are", len(self.validationRecords), "videos for validation.")
      print("[INFO] There are", len(self.testRecords), "videos for testing.")
      print("[COMPLETED] ReadVideosMetadata.")
    
  def ManipulateVideosToFrames(self, which):
    if(self.debug): print("[STARTED] ManipulateVideosToFrames.")
    
    if (which == "train"):
      records = self.trainRecords
    elif(which == "test"):
      records = self.testRecords
    elif(which == "validation"):
      records = self.validationRecords
    else:
      return
    folder = self.framesDir
    
    if (os.path.exists(folder)): shutil.rmtree(folder)
    if (not os.path.exists(folder)): os.mkdir(folder)
    if(self.debug): print("[INFO] Starting to work on [%s]." % which)

    for i, record in enumerate(records):
      columns = ["number", "video_name", "class_name", "g", "c", "g_c", "extension"]
      frames = []
      classPath = os.path.join(folder, record[2])
      gcPath = os.path.join(classPath, record[5])
      if (not os.path.exists(classPath)): os.mkdir(classPath)
      if (not os.path.exists(gcPath)): os.mkdir(gcPath)      
      videoPath = os.path.join(self.path, record[1])
      video = cv2.VideoCapture(videoPath)
      success, img = video.read()
      count = 0
      while success:
        storePath = os.path.join(gcPath, 
          ("frame_%s_%d_%d.jpg" % (record[5], i + 1, count + 1)))
        cv2.imwrite(storePath, img)   
        success, img = video.read()
        count += 1
      print("[INFO] [%d/%d] %s has completed processing with [%d] frames." % \
            (i+1, len(records), record[1], count))
    if(self.debug): print("[COMPLETED] ManipulateVideosToFrames.")

        
  @threadsafe_generator
  def LiveGeneratorImproved(self, which, subWhich, noOfThreads=5,
      doAugmentation=True, rotate=10, shearRange=5,
      zoom=5, horizontalFlip=True, translationRange=5, 
      changeBrightness=True):
    if (self.debug): print("[STARTED] LiveGeneratorImproved")

    if (which == "train"): 
        records = self.trainRecords
    elif(which == "test"): 
        records = self.testRecords
    elif(which == "validation"): 
        records = self.validationRecords
    elif(which == "all"): 
        records = self.records
    else: return
    frames = self.framesDir
    flows = self.flowsDir
    
    whichElement = which
    subWhichElement = subWhich
    
    random.shuffle(records)
    recordsTemp = records[:]
    if (self.debug): print("Working on:", which)
    
    class Data(object):
        def __init__(self, records, recordsTemp):
            self.XY = []
            self.records = records
            self.recordsTemp = recordsTemp
    
    XYobj = Data(records, recordsTemp)

    class myThread (threading.Thread):
      def __init__(self, XYobj, frames, flows, whichElement, 
                  subWhichElement, classes, isGray,
                  inputShape, doAugmentation,
                  rotate, shearRange, zoom, horizontalFlip, 
                  translationRange, changeBrightness, shuffle,
                  maxInMemory, segmentation, packets):
        threading.Thread.__init__(self)
        self.XYobj = XYobj
        self.classes = classes
        self.doAugmentation = doAugmentation
        self.rotate = rotate
        self.zoom = zoom
        self.shearRange = shearRange
        self.horizontalFlip = horizontalFlip
        self.changeBrightness = changeBrightness
        self.translationRange = translationRange
        self.inputShape = inputShape
        self.frames = frames
        self.flows = flows
        self.whichElement = whichElement
        self.subWhichElement = subWhichElement
        self.isGray = isGray
        self.shuffle = shuffle
        self.maxInMemory = maxInMemory
        self.segmentation = segmentation
        self.packets = packets
        
      def run(self):
        columns = ["number", "video_name", "class_name", "g", "c", "g_c", "extension"]
        while(True):
          while (len(XYobj.XY) > self.maxInMemory):
            time.sleep(random.random())
            continue
          if (len(XYobj.recordsTemp) <= 0):
            print("[INFO] Filling the records again for %s." % self.whichElement)
            XYobj.recordsTemp = XYobj.records[:]
            if (self.shuffle): random.shuffle(XYobj.recordsTemp)
          try:
            
            if (self.subWhichElement[0] == "flows"):
              record = XYobj.recordsTemp.pop(0)  
              videoName = record[1]
              className = record[2]
              videoNoExtName = record[1].split(".")[0]
              uFolderPath = os.path.join(flows, "u", videoNoExtName) 
              vFolderPath = os.path.join(flows, "v", videoNoExtName) 
              classIndex = self.classes.index(className)
              oneHotClass = to_categorical(classIndex, len(self.classes))
              assert oneHotClass.shape[0] == len(self.classes)
            
              uDirFrames = os.listdir(uFolderPath)
              Lu = len(uDirFrames)
              vDirFrames = os.listdir(vFolderPath)
              Lv = len(vDirFrames)
            
              if (self.subWhichElement[1] == "single"):
                for j in range(self.packets):
                  start = random.randint(0, Lu - int(self.segmentation / 2))
                  if (start <= 0): start = 0
                  stack = []
                  for i in range(int(self.segmentation / 2)):
                    uImagePath = os.path.join(uFolderPath, uDirFrames[i+start])
                    vImagePath = os.path.join(vFolderPath, vDirFrames[i+start])
                    if (self.isGray): 
                      uFrame = cv2.imread(uImagePath, cv2.IMREAD_GRAYSCALE)
                      vFrame = cv2.imread(vImagePath, cv2.IMREAD_GRAYSCALE)
                    else: 
                      uFrame = cv2.imread(uImagePath)
                      vFrame = cv2.imread(vImagePath)
                    uFrame = cv2.resize(uFrame, self.inputShape[:2], interpolation=cv2.INTER_AREA)
                    vFrame = cv2.resize(vFrame, self.inputShape[:2], interpolation=cv2.INTER_AREA)
                    stack.append(uFrame) 
                    stack.append(vFrame)
                  stack = np.array(stack)
                  stack = stack.astype('float32') / 255.0
                  stack = np.swapaxes(stack, 0, 1)
                  stack = np.swapaxes(stack, 1, 2)
                  XYobj.XY.append((stack, oneHotClass))
              elif (self.subWhichElement[1] == "stacked"):
                overallStack = []
                for j in range(self.packets):
                  start = random.randint(0, Lu - int(self.segmentation / 2))
                  if (start <= 0): start = 0
                  stack = []
                  for i in range(int(self.segmentation / 2)):
                    uImagePath = os.path.join(uFolderPath, uDirFrames[i+start])
                    vImagePath = os.path.join(vFolderPath, vDirFrames[i+start])
                    if (self.isGray): 
                      uFrame = cv2.imread(uImagePath, cv2.IMREAD_GRAYSCALE)
                      vFrame = cv2.imread(vImagePath, cv2.IMREAD_GRAYSCALE)
                    else: 
                      uFrame = cv2.imread(uImagePath)
                      vFrame = cv2.imread(vImagePath)
                    uFrame = cv2.resize(uFrame, self.inputShape[:2], interpolation=cv2.INTER_AREA)
                    vFrame = cv2.resize(vFrame, self.inputShape[:2], interpolation=cv2.INTER_AREA)
                    stack.append(uFrame) 
                    stack.append(vFrame)
                  stack = np.array(stack)
                  stack = stack.astype('float32') / 255.0
                  stack = np.swapaxes(stack, 0, 1)
                  stack = np.swapaxes(stack, 1, 2)
                  overallStack.append(stack)
                XYobj.XY.append((overallStack, oneHotClass))
            
            elif (self.subWhichElement[0] == "frames"):
              record = XYobj.recordsTemp.pop(0)  
              className = record[2]
              gcName = record[5]
              folderPath = os.path.join(frames, className, gcName)    
              classIndex = self.classes.index(className)
              oneHotClass = to_categorical(classIndex, len(self.classes))
              assert oneHotClass.shape[0] == len(self.classes)

              dirFrames = os.listdir(folderPath)
              L = len(dirFrames)
              if (self.subWhichElement[1] == "single"):
                if (self.packets > int(L / self.segmentation)):
                    self.packets = int(L / self.segmentation)
                for offset in range(self.packets):
                  for i in range(self.segmentation):
                    imagePath = os.path.join(folderPath, dirFrames[i + self.segmentation * offset])
                    if (self.isGray): frame = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
                    else: frame = cv2.imread(imagePath)
                    frame = cv2.resize(frame, self.inputShape[:2], interpolation=cv2.INTER_AREA)
                    XYobj.XY.append((frame, oneHotClass))

                    if (self.doAugmentation):
                      augFrame = AugmentFrame(frame, self.inputShape,
                        self.rotate, self.shearRange, 
                        self.zoom, self.horizontalFlip, 
                        self.translationRange, self.changeBrightness)
                      augFrame = cv2.resize(augFrame, self.inputShape[:2],
                          interpolation=cv2.INTER_AREA)
                      XYobj.XY.append((augFrame, oneHotClass))
              elif (self.subWhichElement[1] == "stacked"):
                if (self.packets > L - self.segmentation):
                    self.packets = L - self.segmentation
                for offset in range(self.packets):
                  stack = []
                  for i in range(self.segmentation):
                    imagePath = os.path.join(folderPath, dirFrames[i + offset])
                    if (self.isGray): frame = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
                    else: frame = cv2.imread(imagePath)
                    frame = cv2.resize(frame, self.inputShape[:2], interpolation=cv2.INTER_AREA)
                    stack.append(frame)  
                  stack = np.array(stack)
                  stack = stack.astype('float32') / 255.0
                  XYobj.XY.append((stack, oneHotClass))
          except Exception as e:
            print("\nError:", e)
            pass

    threads = []
    if (len(threads) <= 0):
      for i in range(noOfThreads):
        threads.append(myThread(XYobj, frames, flows, 
          whichElement, subWhichElement, self.classes,
          self.isGray, self.inputShape,
          doAugmentation, rotate, shearRange, zoom, horizontalFlip, 
          translationRange, changeBrightness, 
          self.shuffle, self.maxInMemory, self.segmentation, self.packets))
      for i in range(noOfThreads):
        print("Starting " + str(i+1) + " Thread!")
        threads[i].start()

    while(1):
      if (len(XYobj.XY) >= self.batchSize):
        XYt = XYobj.XY[:self.batchSize]
        XYobj.XY = XYobj.XY[self.batchSize:]
        if(self.shuffle):
          random.shuffle(XYobj.XY)
          random.shuffle(XYt)
        Xt = [el[0] for el in XYt]
        Xt = np.array(Xt)
        if (subWhichElement[0] == "frames" and subWhichElement[1] == "single"):
          Xt = np.array(Xt).reshape(-1, self.inputShape[0], 
            self.inputShape[1], self.inputShape[2])
          Xt = Xt.astype('float32') / 255.0
          
        yt = [el[1] for el in XYt]
        yt = np.array(yt)
        yt = np.squeeze(yt)

        yield Xt, yt
    if (self.debug): print("[COMPLETED] LiveGeneratorImproved")     

# Models Class

In [ ]:
class Models(object):
  def __init__(self, inputShape, noOfClasses,
               hiddenActivation, outputActivation,
               optimizer, weightsInit, batchSize, noOfEpochs,
               loss, metrics, stepsPerEpoch, validationSteps,
               loadModel=False,
               datasetKeyword=None,
               resultsFolder=None, modelName=None,
               savedFolder=None, savedHDF5=None, 
               weights='imagenet', segmentation=1, packets=1):
    self.inputShape = inputShape
    self.hiddenActivation = hiddenActivation
    self.outputActivation = outputActivation
    self.weightsInit = weightsInit
    self.noOfClasses = noOfClasses
    self.loadModel = loadModel
    self.resultsFolder = resultsFolder
    self.savedFolder = savedFolder
    self.savedHDF5 = savedHDF5
    self.modelName = modelName
    self.optimizer = optimizer
    self.batchSize = batchSize
    self.noOfEpochs = noOfEpochs
    self.stepsPerEpoch = stepsPerEpoch
    self.validationSteps = validationSteps
    self.loss = loss
    self.metrics = metrics
    self.model = None
    self.datasetKeyword = datasetKeyword
    self.weights = weights
    self.segmentation = segmentation
    self.packets = packets

    assert self.modelName is not None
    assert self.datasetKeyword is not None
    assert self.resultsFolder is not None
    assert self.noOfClasses > 0

    if (self.loadModel): 
        self.model = self.LoadModel()
        self.time = self.savedFolder.split("-")[-1]
    else:
      if (self.modelName == "SpatialCNNnLSTM"): self.model = self.SpatialCNNnLSTM()
      elif (self.modelName == "TemporalCNNnLSTM"): self.model = self.TemporalCNNnLSTM()
      elif (self.modelName == "TemporalCNNModel"): self.model = self.TemporalCNNModel()
      elif (self.modelName == "DenseNet201Model"): self.model = self.DenseNet201Model()
      elif (self.modelName == "SpatialSmallCNNnLSTM"): self.model = self.SpatialSmallCNNnLSTM()
      
      self.time = time.strftime("%y%m%d%H%M", time.localtime())
      self.savedFolder = os.path.join(self.resultsFolder, 
        self.datasetKeyword + "-" + self.modelName + "-" + self.time)
      if not os.path.exists(self.savedFolder):
        print("Creating:", self.savedFolder)
        os.makedirs(self.savedFolder)
      else: print("Created Previously:", self.savedFolder)
      
    self.PrepareCallbacks()

  def PrepareCallbacks(self):
    # Callbacks: Save the model.
    filepath = "{}-{}-{}".format(self.datasetKeyword, self.modelName, self.time)
    filepath = filepath + "-{epoch:03d}-{val_accuracy:.4f}-{val_loss:.3f}.hdf5"

    class TrainHistory(tensorflow.keras.callbacks.Callback):
      def __init__(self, obj, logs={}):
        self.logs = []
        self.obj = obj
      def on_train_begin(self, logs=None):
        self.startTime = time.time()
      def on_train_end(self, logs=None):
        self.endTime = time.time()
        s = str(self.startTime) + "\n"
        s += str(self.endTime) + "\n"
        deltaTime = self.endTime - self.startTime
        hours, rem = divmod(deltaTime, 3600)
        minutes, seconds = divmod(rem, 60)
        s += "{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds)
        f = open(os.path.join(self.obj.savedFolder, 
                  "time-" + self.obj.time + ".data"), "a") 
        f.write("%s\n" % s)
        f.close()
      def on_epoch_end(self, epoch, logs={}):
        obj = self.obj
        data = [obj.datasetKeyword, obj.modelName, 
                str(obj.time), str(time.time()),
                obj.weightsInit, obj.optimizer, 
                str(epoch), str(obj.noOfEpochs),
                str(obj.inputShape[0]), str(obj.inputShape[1]), 
                obj.hiddenActivation, obj.outputActivation, 
                str(obj.noOfClasses), 
                str(obj.batchSize), obj.loss,
                str(obj.stepsPerEpoch), str(obj.validationSteps)]
        for k in logs.keys():
          data.append(str(logs[k]))
        s = ','.join(data)
        f = open(os.path.join(obj.savedFolder, 
                              "history-" + obj.time + ".csv"), "a") 
        f.write("%s\n" % s)
        f.close()

    chkValAcc = ModelCheckpoint(
      filepath=os.path.join(self.savedFolder, filepath),
      verbose=1,
      monitor='val_accuracy',
      save_best_only=True
    )

    chkValLoss = ModelCheckpoint(
      filepath=os.path.join(self.savedFolder, filepath),
      verbose=1,
      monitor='val_loss',
      save_best_only=True
    )
    
    tb = TensorBoard(log_dir=os.path.join(self.savedFolder))
    
    reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.25,
      patience=5, verbose=1, min_lr=0.001)

    # Learning rate schedule.
    lrSchedule = LearningRateScheduler(scheduler, verbose=1)
    
    # Callbacks: Early stoper.
    earlyStopper = EarlyStopping(monitor='accuracy', patience=100)

    # Callbacks: Save results.
    csvLogger = CSVLogger(os.path.join(self.savedFolder, 'log-' + \
        str(self.time) + '.csv'), append=True)

    self.callbacks = [earlyStopper, csvLogger, tb, reduceLR, lrSchedule,
                      chkValAcc, chkValLoss, TrainHistory(self)]

  def LoadModel(self):
    path = self.savedFolder
    hdf5 = os.path.join(path, self.savedHDF5)
    assert os.path.exists(path)

    dependencies = {
      'recall_m': recall_m,
      'precision_m': precision_m,
      'f1_m': f1_m,
    }
    return load_model(hdf5, custom_objects=dependencies)

  def CompileModel(self):
    self.model.compile(
      optimizer=self.optimizer,
      loss=self.loss,
      metrics=self.metrics,
    )

  def RefineCompileModel(self):
    L = int(len(self.model.layers))
    factor = int(2*L/3)
    for layer in self.model.layers[factor:]: layer.trainable = True
    for layer in self.model.layers[:factor]: layer.trainable = False
    self.CompileModel()

  def FitModel(self, trainGenerator, validationGenerator):
    self.model.summary()
    self.model.fit_generator(
      generator=trainGenerator,
      steps_per_epoch=self.stepsPerEpoch,
      validation_data=validationGenerator,
      validation_steps=self.validationSteps,
      epochs=self.noOfEpochs,
      callbacks=self.callbacks,
      max_queue_size=1024,
      use_multiprocessing=False,
      workers=1,
      verbose=1,
    )

  def EvaluateModel(self, testGenerator):
    results = self.model.evaluate_generator(
      generator=testGenerator,
      steps=self.stepsPerEpoch,
      max_queue_size=1024,
      use_multiprocessing=False,
      workers=1,
      verbose=1,
    )
    print(self.model.metrics_names)
    print(results)

  def DenseNet201Model(self):
    baseModel = DenseNet201(weights=self.weights, include_top=False)
    x = baseModel.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=self.hiddenActivation)(x) # FC
    predictions = Dense(self.noOfClasses, activation=self.outputActivation)(x)
    model = Model(inputs=baseModel.input, outputs=predictions)
    L = int(len(model.layers))
    for layer in model.layers[:]: layer.trainable = False
    for layer in model.layers[L-4:]: layer.trainable = True
    return  model

  def TemporalCNNModel(self):
    p = self.inputShape
    shape = [p[0], p[1], self.segmentation]
    model = Sequential()
    model.add(Conv2D(96, (7, 7), strides=2, padding='same', input_shape=shape))
    model.add(BatchNormalization())
    model.add(Activation(self.hiddenActivation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(256, (5, 5), strides=2, padding='same'))
    model.add(Activation(self.hiddenActivation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(512, (3, 3), strides=1, activation=self.hiddenActivation, padding='same'))
    model.add(Conv2D(512, (3, 3), strides=1, activation=self.hiddenActivation, padding='same'))
    model.add(Conv2D(512, (3, 3), strides=1, activation=self.hiddenActivation, padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(4096, activation=self.hiddenActivation))
    model.add(Dropout(0.9))
    model.add(Dense(2048, activation=self.hiddenActivation))
    model.add(Dropout(0.9))
    model.add(Dense(self.noOfClasses, activation=self.outputActivation))
    return  model

  def SpatialSmallCNNnLSTM(self):
    p = self.inputShape
    shape = [self.segmentation, p[0], p[1], p[2]]
    model = Sequential()
    model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001)),
      input_shape=shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(Conv2D(32, (3,3), 
      kernel_initializer=self.weightsInit,
      kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
    model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
    model.add(TimeDistributed(Conv2D(256, (3, 3), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(Conv2D(256, (3, 3), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
    model.add(TimeDistributed(Conv2D(512, (3, 3), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(Conv2D(512, (3, 3), padding='same',
      kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation(self.hiddenActivation)))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(Flatten()))
    model.add(CuDNNLSTM(256, return_sequences=False))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))  
    model.add(Dense(self.noOfClasses, activation=self.outputActivation))
    return model

  def TemporalCNNnLSTM(self):
    p = self.inputShape
    shape = [self.packets, p[0], p[1], self.segmentation]
    return self.CNNnLSTM(shape)

  def SpatialCNNnLSTM(self):
    p = self.inputShape
    shape = [self.segmentation, p[0], p[1], p[2]]
    return self.CNNnLSTM(shape)

  def CNNnLSTM(self, shape):
      model = Sequential()
      model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001)),
        input_shape=shape))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(Conv2D(32, (3,3), 
        kernel_initializer=self.weightsInit,
        kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

      model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

      model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

      model.add(TimeDistributed(Conv2D(256, (3, 3), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(Conv2D(256, (3, 3), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

      model.add(TimeDistributed(Conv2D(512, (3, 3), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(Conv2D(512, (3, 3), padding='same',
        kernel_initializer=self.weightsInit, kernel_regularizer=l2(l=0.001))))
      model.add(TimeDistributed(BatchNormalization()))
      model.add(TimeDistributed(Activation(self.hiddenActivation)))
      model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

      model.add(TimeDistributed(Flatten()))
      model.add(LSTM(256, return_sequences=False, dropout=0.5))
      model.add(Dense(self.noOfClasses, activation=self.outputActivation))
      return model


# Working

In [ ]:
# Constants
IMAGE_SHAPE_64 = (64, 64, 3)
IMAGE_SHAPE_80 = (80, 80, 3)
IMAGE_SHAPE_100 = (100, 100, 3)
IMAGE_SHAPE_128 = (128, 128, 3)
IMAGE_SHAPE_256 = (256, 256, 3)
GRAY_SHAPE_64 = (64, 64, 1)
GRAY_SHAPE_100 = (100, 100, 1)
GRAY_SHAPE_128 = (128, 128, 1)
GRAY_SHAPE_256 = (256, 256, 1)
WEIGHTS_IMAGENET = "imagenet"
OUTPUT_ACTIVATION_SOFTMAX = "softmax"
HIDDEN_ACTIVATION_RELU = "relu"
LOSS = "categorical_crossentropy"
RMS_OPTIMIZER = "rmsprop"
ADAM_OPTIMIZER = "adam"
NADAM_OPTIMIZER = "nadam"
ADAGRAD_OPTIMIZER = "adagrad"
ADAMAX_OPTIMIZER = "adamax"
ADADELTA_OPTIMIZER = "adadelta"
SGD_OPTIMIZER = 'sgd'
METRICS = ['accuracy', 'top_k_categorical_accuracy', 
           recall_m, precision_m, f1_m]
STEPS_PER_EPOCH_16384 = 16384
STEPS_PER_EPOCH_4096 = 4096
STEPS_PER_EPOCH_1024 = 1024
STEPS_PER_EPOCH_768 = 768
STEPS_PER_EPOCH_512 = 512
STEPS_PER_EPOCH_256 = 256
STEPS_PER_EPOCH_128 = 128
STEPS_PER_EPOCH_64 = 64
STEPS_PER_EPOCH_32 = 32
VALIDATION_STEPS_1024 = 1024
VALIDATION_STEPS_512 = 512
VALIDATION_STEPS_256 = 256
VALIDATION_STEPS_128 = 128
VALIDATION_STEPS_64 = 64
VALIDATION_STEPS_32 = 32
NO_OF_EPOCHS_4 = 4
NO_OF_EPOCHS_8 = 8
NO_OF_EPOCHS_16 = 16
NO_OF_EPOCHS_32 = 32
NO_OF_EPOCHS_64 = 64
NO_OF_EPOCHS_128 = 128
NO_OF_EPOCHS_150 = 150
NO_OF_EPOCHS_256 = 256
NO_OF_EPOCHS_512 = 512
BATCH_SIZE_2048 = 2048
BATCH_SIZE_1024 = 1024
BATCH_SIZE_512 = 512
BATCH_SIZE_256 = 256
BATCH_SIZE_128 = 128
BATCH_SIZE_64 = 64
BATCH_SIZE_32 = 32
BATCH_SIZE_16 = 16
BATCH_SIZE_8 = 8
BATCH_SIZE_4 = 4
BATCH_SIZE_2 = 2
LIMIT_SEGMENTATION_1 = 1
LIMIT_SEGMENTATION_2 = 2
LIMIT_SEGMENTATION_4 = 4
LIMIT_SEGMENTATION_8 = 8
LIMIT_SEGMENTATION_10 = 10
LIMIT_SEGMENTATION_16 = 16
LIMIT_SEGMENTATION_20 = 20
LIMIT_SEGMENTATION_25 = 25
LIMIT_SEGMENTATION_32 = 32
LIMIT_SEGMENTATION_64 = 64
LIMIT_SEGMENTATION_128 = 128
LIMIT_SEGMENTATION_256 = 256
SPLITS_70_15_15 = (0.70, 0.15, 0.15)
SPLITS_80_10_10 = (0.80, 0.10, 0.10)
OPTIMIZERS = ['adamax', 'sgd', 'adam', 'nadam', 'adadelta', 'adagrad']
WEIGHTS_INITS = ['he_normal', 'he_uniform', 'glorot_normal', 'glorot_uniform',
    'lecun_normal', 'lecun_uniform'
]

In [ ]:
# Configurations
PROJECT_FOLDER = ""
DATASET_FOLDER = "UCF_Videos"
RESULTS_FOLDER = DATASET_FOLDER + "_Results"
PROJECT_FOLDER = os.path.join(os.getcwd(), PROJECT_FOLDER)
DATASET_FOLDER = os.path.join(PROJECT_FOLDER, DATASET_FOLDER)
RESULTS_FOLDER = os.path.join(PROJECT_FOLDER, RESULTS_FOLDER)
DATASET_KEYWORD = "UCF"
OPTIMIZER = OPTIMIZERS[0]
WEIGHTS_INIT = WEIGHTS_INITS[3]
SPLITS = SPLITS_70_15_15
HIDDEN_ACTIVATION = HIDDEN_ACTIVATION_RELU
OUTPUT_ACTIVATION = OUTPUT_ACTIVATION_SOFTMAX
IMAGE_SHAPE = IMAGE_SHAPE_100
BATCH_SIZE = BATCH_SIZE_32
NO_OF_EPOCHS = NO_OF_EPOCHS_16
STEPS_PER_EPOCH = STEPS_PER_EPOCH_16384
VALIDATION_STEPS = VALIDATION_STEPS_1024
LOAD_MODEL = False
SAVED_FOLDER = ""
SAVED_FOLDER = os.path.join(RESULTS_FOLDER, SAVED_FOLDER)
SAVED_HDF5 = ""
DEBUG = True
SHUFFLE = True
AUGMENT = True
MAX_IN_MEMORY = 10000
IS_GRAY = False
MODEL_NAME = "SpatialSmallCNNnLSTM" 
LIMIT_SEGMENTATION = LIMIT_SEGMENTATION_10
PACKETS = 1000 # Number of times per record
SUB_WHICH = ["frames", "stacked"]
WEIGHTS = WEIGHTS_IMAGENET

In [ ]:
lenContent = len(os.listdir(DATASET_FOLDER))
print("Number of items in the folder =", lenContent)

## VideosDataset Class Usage

In [ ]:
print(DATASET_FOLDER)
ucf = VideosDataset(
  DATASET_FOLDER, storageDir=PROJECT_FOLDER,
  debug=DEBUG, isGray=IS_GRAY, splits=SPLITS, 
  batchSize=BATCH_SIZE, inputShape=IMAGE_SHAPE,
  datasetKeyword=DATASET_KEYWORD, storeCSV=True,
  maxInMemory=MAX_IN_MEMORY, shuffle=SHUFFLE,
  segmentation=LIMIT_SEGMENTATION, packets=PACKETS
)
ucf.ReadVideosMetadata()
trainGenerator = ucf.LiveGeneratorImproved('train', SUB_WHICH, 
  noOfThreads=128, doAugmentation=AUGMENT, 
  rotate=10, shearRange=5, zoom=5, 
  horizontalFlip=True, translationRange=5, 
  changeBrightness=True
)
validationGenerator = ucf.LiveGeneratorImproved('validation', SUB_WHICH, 
  noOfThreads=32, doAugmentation=False, 
  rotate=10, shearRange=5, zoom=5, 
  horizontalFlip=True, translationRange=5, 
  changeBrightness=True
)
noOfClasses = ucf.GetNoOfClasses()

## Models Class Usage

In [ ]:
models = Models(IMAGE_SHAPE, noOfClasses,
  HIDDEN_ACTIVATION, OUTPUT_ACTIVATION, 
  OPTIMIZER, WEIGHTS_INIT, BATCH_SIZE, NO_OF_EPOCHS,
  LOSS, METRICS, STEPS_PER_EPOCH, VALIDATION_STEPS,
  loadModel=LOAD_MODEL, datasetKeyword=DATASET_KEYWORD,
  resultsFolder=RESULTS_FOLDER, modelName=MODEL_NAME,
  savedFolder=SAVED_FOLDER, savedHDF5=SAVED_HDF5,
  weights=WEIGHTS, segmentation=LIMIT_SEGMENTATION,
  packets=PACKETS
)

models.CompileModel()
models.FitModel(trainGenerator, validationGenerator)

del models